In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Bile_Duct_Cancer/GSE131027'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"High frequency of pathogenic germline variants in genes associated with homologous recombination repair in patients with advanced solid cancers"
!Series_summary	"We identified pathogenic and likely pathogenic variants in 17.8% of the patients within a wide range of cancer types. In particular, mesothelioma, ovarian cancer, cervical cancer, urothelial cancer, and cancer of unknown primary origin displayed high frequencies of pathogenic variants. In total, 22 BRCA1 and BRCA2 germline variant were identified in 12 different cancer types, of which 10 (45%) variants were not previously identified in these patients. Pathogenic germline variants were predominantly found in DNA repair pathways; approximately half of the variants were within genes involved in homologous recombination repair. Loss of heterozygosity and somatic second hits were identified in several of these genes, supporting possible causality for cancer development. A potential treatment t

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Check if the dataset contains gene expression data
is_gene_available = True

# Step 2: Data Availability and Data Type Conversion

sample_characteristics = {
    0: ['tissue: tumor biopsy'],
    1: ['cancer: Breast cancer', 'cancer: Colorectal cancer', 'cancer: Bile duct cancer', 'cancer: Mesothelioma', 'cancer: Urothelial cancer', 'cancer: Pancreatic cancer', 'cancer: Melanoma', 'cancer: Hepatocellular carcinoma', 'cancer: Ovarian cancer', 'cancer: Cervical cancer', 'cancer: Head and Neck cancer', 'cancer: Sarcoma', 'cancer: Prostate cancer', 'cancer: Adenoid cystic carcinoma', 'cancer: NSCLC', 'cancer: Oesophageal cancer', 'cancer: Thymoma', 'cancer: Others', 'cancer: CUP', 'cancer: Renal cell carcinoma', 'cancer: Gastric cancer', 'cancer: Neuroendocrine cancer', 'cancer: vulvovaginal'],
    2: ['mutated gene: ATR', 'mutated gene: FAN1', 'mutated gene: ERCC3', 'mutated gene: FANCD2', 'mutated gene: BAP1', 'mutated gene: DDB2', 'mutated gene: TP53', 'mutated gene: ATM', 'mutated gene: CHEK1', 'mutated gene: BRCA1', 'mutated gene: WRN', 'mutated gene: CHEK2', 'mutated gene: BRCA2', 'mutated gene: XPC', 'mutated gene: PALB2', 'mutated gene: ABRAXAS1', 'mutated gene: NBN', 'mutated gene: BLM', 'mutated gene: FAM111B', 'mutated gene: FANCA', 'mutated gene: MLH1', 'mutated gene: BRIP1', 'mutated gene: IPMK', 'mutated gene: RECQL', 'mutated gene: RAD50', 'mutated gene: FANCM', 'mutated gene: GALNT12', 'mutated gene: SMAD9', 'mutated gene: ERCC2', 'mutated gene: FANCC'],
    3: ['predicted: HRDEXP: HRD', 'predicted: HRDEXP: NO_HRD'],
    4: ['parp predicted: kmeans-2: PARP sensitive', 'parp predicted: kmeans-2: PARP insensitive']
}

# Check for 'Bile_Duct_Cancer' data availability
for key, values in sample_characteristics.items():
    if 'cancer: Bile duct cancer' in values:
        trait_row = key
        break

# Check for 'age' data availability
# Explicit check is required but as this information is not found in the given dictionary, it should be confirmed not available
age_row = None

# Check for 'gender' data availability
# Explicit check is required but as this information is not found in the given dictionary, it should be confirmed not available
gender_row = None

# Define conversion functions
def convert_trait(value):
    if 'Bile duct cancer' in value:
        return 1
    else:
        return 0

def convert_age(value):
    try:
        return float(value.split(':')[-1].strip())
    except ValueError:
        return None

def convert_gender(value):
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower():
        return 1
    else:
        return None

# Update the save_cohort_info function
save_cohort_info('GSE131027', './preprocessed/Bile_Duct_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Bile_Duct_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Bile_Duct_Cancer/trait_data/GSE131027.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3759992': [0], 'GSM3759993': [0], 'GSM3759994': [0], 'GSM3759995': [1], 'GSM3759996': [0], 'GSM3759997': [0], 'GSM3759998': [0], 'GSM3759999': [0], 'GSM3760000': [0], 'GSM3760001': [0], 'GSM3760002': [0], 'GSM3760003': [0], 'GSM3760004': [0], 'GSM3760005': [0], 'GSM3760006': [0], 'GSM3760007': [0], 'GSM3760008': [0], 'GSM3760009': [0], 'GSM3760010': [0], 'GSM3760011': [0], 'GSM3760012': [0], 'GSM3760013': [0], 'GSM3760014': [0], 'GSM3760015': [0], 'GSM3760016': [0], 'GSM3760017': [0], 'GSM3760018': [0], 'GSM3760019': [0], 'GSM3760020': [0], 'GSM3760021': [0], 'GSM3760022': [0], 'GSM3760023': [0], 'GSM3760024': [0], 'GSM3760025': [0], 'GSM3760026': [0], 'GSM3760027': [0], 'GSM3760028': [1], 'GSM3760029': [0], 'GSM3760030': [0], 'GSM3760031': [0], 'GSM3760032': [0], 'GSM3760033': [0], 'GSM3760034': [0], 'GSM3760035': [0], 'GSM3760036': [1], 'GSM3760037': [0], 'GSM3760038': [0], 'GSM3760039': [0], 'GSM3760040': [0], 'GSM3760041': [0], 'GSM3760042': [0], 'GSM3760043': [0], 'GSM3760044

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for probe IDs (identifiers) and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Bile_Duct_Cancer/gene_data/GSE131027.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Bile_Duct_Cancer')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE131027', './preprocessed/Bile_Duct_Cancer/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Bile_Duct_Cancer/GSE131027.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Bile_Duct_Cancer', the least common label is '1.0' with 5 occurrences. This represents 5.43% of the dataset.
The distribution of the feature 'Bile_Duct_Cancer' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Bile_Duct_Cancer/cohort_info.json
